In [2]:
import scipy.io
import numpy as np
# from functions_based import *
import knots_ML.dots_processing as dp
from plotly.offline import plot

In [3]:
def mat_to_np(file, variable=None):
    mat = scipy.io.loadmat(file)
    if variable is None:
        print(mat.keys())
        return

    numpy_array = mat[variable]
    return numpy_array

def make_dots_from_array(A):
    dots = []
    for z, z_array in enumerate(A):
        for x, y in z_array:
            dots.append([x, y, z])
    return np.array(dots)


In [4]:
def make_dots(file, variable=None):
    array = mat_to_np(file, variable)[0]
    dots = make_dots_from_array(array)
    return dots

def scale_to_range(column, new_min, new_max):
    min_val = column.min()
    max_val = column.max()
    scaled_column = (column - min_val) / (max_val - min_val) * (new_max - new_min) + new_min
    return scaled_column


def round_dots_same_as_z(dots):
    max_z = (dots[:, 2]).max()
    min_z = (dots[:, 2]).min()
    scaled_first_column = scale_to_range(dots[:, 0], min_z, max_z)
    scaled_second_column = scale_to_range(dots[:, 1], min_z, max_z)
    rounded_first_column = np.rint(scaled_first_column)
    rounded_second_column = np.rint(scaled_second_column)
    rounded_data = np.column_stack((rounded_first_column, rounded_second_column, dots[:, 2]))
    return rounded_data

def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))

def sort_points_by_proximity(points):

    # Start with the first point
    points = np.array(points)
    sorted_points = [points[0, :].tolist()]
    points = np.delete(points, 0, axis=0)

    while len(points) > 0:
        last_point = sorted_points[-1]
        distances = np.linalg.norm(points - last_point, axis=1)
        closest_idx = np.argmin(distances)
        sorted_points.append(points[closest_idx, :].tolist())
        points = np.delete(points, closest_idx, axis=0)

    return np.array(sorted_points)

In [5]:
trefoil_standard_path = './experimental_data/dots_coord.mat'
variable = 'p_charges'

trefoil_standard_not_r = make_dots(trefoil_standard_path, variable)

In [6]:
trefoil_standard_not_sorted = round_dots_same_as_z(trefoil_standard_not_r)
trefoil_standard = sort_points_by_proximity(trefoil_standard_not_sorted)
trefoil_standard

array([[15., 33.,  0.],
       [15., 33.,  1.],
       [16., 33.,  2.],
       [16., 34.,  3.],
       [16., 34.,  4.],
       [16., 35.,  5.],
       [17., 34.,  6.],
       [16., 36.,  7.],
       [16., 37.,  8.],
       [16., 38.,  9.],
       [16., 38., 10.],
       [17., 38., 11.],
       [17., 38., 12.],
       [17., 37., 13.],
       [17., 37., 14.],
       [17., 37., 15.],
       [17., 37., 16.],
       [16., 37., 17.],
       [17., 38., 18.],
       [17., 37., 19.],
       [17., 37., 20.],
       [17., 37., 21.],
       [17., 37., 22.],
       [17., 37., 23.],
       [16., 38., 24.],
       [16., 39., 25.],
       [17., 40., 26.],
       [16., 40., 27.],
       [17., 40., 28.],
       [17., 41., 29.],
       [16., 41., 30.],
       [16., 42., 31.],
       [17., 43., 32.],
       [17., 43., 33.],
       [18., 42., 34.],
       [18., 42., 35.],
       [18., 42., 36.],
       [18., 42., 37.],
       [19., 42., 38.],
       [19., 43., 39.],
       [19., 43., 40.],
       [20., 43.

In [9]:
dotSize = 10
boundary_3D_k = [[0, 0, 0], [50, 50, 50]]
labels = [str(i) for i in range(trefoil_standard.shape[0])]
# fig = dp.plotDots_Hopf(dots_init, boundary_3D_k, color='red', show=False, size=dotSize)



In [12]:
fig = dp.plotDots_Hopf(trefoil_standard, boundary_3D_k, color='royalblue', show=False, size=dotSize, text=labels)
plot(fig, filename='my_plot.html')

'my_plot.html'

In [25]:
# indices_line1 = np.r_[207:213+1, 214:261+1, 71:140+1]
indices_line1 = np.r_[71:149+1, 174:183+1, 267, ]
indices_line2 = np.r_[207:213+1, 214:260+1, 65:70+1, 184:205+1]
max_index = len(trefoil_standard)
all_indices = np.arange(max_index)
set_line1 = set(indices_line1)
set_line2 = set(indices_line2)
set_all = set(all_indices)
set_extra = set(np.array([263, 264, 265, 266, 206, 268, 269, 270, 271]))
indices_line3 = np.array(list(set_all - set_line1 - set_line2 - set_extra))
fig = dp.plotDots_Hopf(trefoil_standard[indices_line1], boundary_3D_k, color='royalblue', show=False, size=dotSize, text=labels)
dp.plotDots_Hopf(trefoil_standard[indices_line2], boundary_3D_k, color='red', show=False, size=dotSize, text=labels, fig=fig)
dp.plotDots_Hopf(trefoil_standard[indices_line3], boundary_3D_k, color='green', show=False, size=dotSize, text=labels, fig=fig, save='standard_trefoil.html')
plot(fig, filename='my_plot.html')
# [213:207]  [214:261]  [71:140]

'my_plot.html'

In [28]:
trefoil_standard_dict = {
    'all_dots' : trefoil_standard,
    'line1' : indices_line1,
    'line2' : indices_line2,
    'line3' : indices_line3,
}
scipy.io.savemat('trefoil_standard.mat', trefoil_standard_dict)